<a href="https://colab.research.google.com/github/wajidminhas/AI-DREAMER/blob/main/context-openai-sdk/context_openai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
gemini_api_key = userdata.get("GEMINI_API_KEY")

In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool, AsyncOpenAI, OpenAIChatCompletionsModel

@dataclass
class UserInfo:
    name: str
    uid: int
    age: int
    gender: str

external_client = AsyncOpenAI(
    api_key = gemini_api_key,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    openai_client=external_client,
    model="gemini-2.0-flash",
)
@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:
    """

    """
    name = wrapper.context.name
    age = wrapper.context.age
    return f"User is {age} years old"

@function_tool
async def fetch_user_name_gender(wrapper: RunContextWrapper[UserInfo]) -> str:
    """

    """
    name = wrapper.context.name
    gender = wrapper.context.gender
    return f"User {name} is and gender is {gender}"

async def main():
    user_info = UserInfo(name="John", uid=123, age=30 , gender = "male")
    """
    llm will user his knowledge to answer about gender regarding USA name of male and felmale
    """
    agent = Agent[UserInfo](
        name="Assistant",
        tools=[fetch_user_age, fetch_user_name_gender],
        model= model


    )

    result = await Runner.run(
        starting_agent=agent,
        input="What is the name of the user and his gender ",
        context=user_info,
    )

    print(result.final_output)
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())

The user's name is John and his gender is male.

